# A Unified View of Entropy-Regularized Markov Decision Processes:
## Article Review



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from renderer import *
from gridworld import *

import scipy
import scipy.optimize as opt
import pdb

In [2]:
grid1 = [
    ['', '', '', 1],
    ['', 'x', '', -1],
    ['', '', '', '']
]

env = GridWorld(gamma=0.95, grid=grid1)

In [3]:
class Policy(object):
    def __init__(self,pi):
        n_states,n_actions = np.shape(pi)
        self.n_actions = n_actions
        self.n_states = n_states
        self.pi = pi
    def draw_action(self,state):
        u = np.random.rand()
        probas = np.cumsum(self.pi[state,:])
        a = 0
        while (a < self.n_actions-1 and (u > probas[a] or self.pi[state,a]==0)):
            a += 1
        return a

In [4]:
class RLModel:
    def __init__(self, env):
        self.env = env
        
    def initialize_pi(self):
        pi = np.zeros((self.env.n_states,self.env.n_actions))
        for s in range(self.env.n_states):
            actions = self.env.state_actions[s]
            for a in actions:
                pi[s,a] = 1./len(actions)
        return pi
    
    def collect_episodes(self, policy=None, horizon=None, n_episodes=1, render=False):
        paths = []

        for _ in range(n_episodes):
            observations = []
            actions = []
            rewards = []
            next_states = []

            state = self.env.reset()
            for _ in range(horizon):
                action = policy.draw_action(state)
                next_state, reward, terminal = self.env.step(state,action)
                if render:
                    self.env.render()
                observations.append(state)
                actions.append(action)
                rewards.append(reward)
                next_states.append(next_state)
                state = copy.copy(next_state)
                if terminal:
                    # Finish rollout if terminal state reached
                    break
                    # We need to compute the empirical return for each time step along the
                    # trajectory
            paths.append(dict(
                states=np.array(observations),
                actions=np.array(actions),
                rewards=np.array(rewards),
                next_states=np.array(next_states)
            ))
        return paths

    
    def render_policy(self):
        render_policy(self.env,self.policy.pi)

## REPS algorithm
### Pseudocode implementation in docs file

In [21]:
class REPS(RLModel):
    def __init__(self, env, p=3, N=100,K=50,eta=0.1):
        RLModel.__init__(self, env)
        self.p = p
        self.N = N
        self.K = K
        self.eta = eta
    
    def compute_new_policy(self, eta, policy, phi, theta, samples):
        log_new_pi = np.zeros((policy.n_states,policy.n_actions))
        A = np.zeros((policy.n_states,policy.n_actions))
        counter = np.zeros((policy.n_states,policy.n_actions))
        nb_samples = 0
        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            rewards = samples[i]['rewards']
            next_states = samples[i]['next_states']

            for j in range(len(states)):
                A[states[j],actions[j]] += rewards[j] + np.dot(phi[next_states[j],:],theta) - np.dot(phi[states[j],:],theta)
                counter[states[j],actions[j]] += 1
                nb_samples += 1
        for s in range(env.n_states):
            for a in range(env.n_actions):
                if counter[s,a]!=0:
                    A[s,a] /= counter[s,a]
        for s in range(policy.n_states):
            for a in range(policy.n_actions):
                argexpo = np.zeros(policy.n_actions)
                if policy.pi[s,a] == 0:
                    log_new_pi[s,a] = -float('inf')
                else:
                    for b in range(policy.n_actions):
                        argexpo[b] = np.log(policy.pi[s,b]+0.0001) + eta * A[s,b]
                    maxi = np.max(argexpo)
                    log_new_pi[s,a] = argexpo[a] - np.log(np.sum(np.exp(argexpo - maxi))) - maxi
        return(Policy(np.exp(log_new_pi)))


    def g(self, theta, eta, phi, samples):
        res = 0
        A = np.zeros((env.n_states,env.n_actions))
        counter = np.zeros((env.n_states,env.n_actions))
        nb_samples = 0
        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            rewards = samples[i]['rewards']
            next_states = samples[i]['next_states']

            for j in range(len(states)):
                A[states[j],actions[j]] += rewards[j] + np.dot(phi[next_states[j],:],theta) - np.dot(phi[states[j],:],theta)
                counter[states[j],actions[j]] += 1
                nb_samples += 1
        for s in range(env.n_states):
            for a in range(env.n_actions):
                if counter[s,a]!=0:
                    A[s,a] /= counter[s,a]
        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            for j in range(len(states)):
                res += np.exp(eta*A[states[j],actions[j]])
        res /= nb_samples
        return (np.log(res)/eta)

    def Dg(self, theta, eta, phi, samples):
        n_states,p = np.shape(phi)
        numerator = 0
        denominator = 0
        A = np.zeros((env.n_states,env.n_actions))
        D = np.zeros((env.n_states,env.n_actions,p))
        counter = np.zeros((env.n_states,env.n_actions))
        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            rewards = samples[i]['rewards']
            next_states = samples[i]['next_states']

            for j in range(len(states)):
                A[states[j],actions[j]] += rewards[j] + np.dot(phi[next_states[j],:],theta) - np.dot(phi[states[j],:],theta)
                D[states[j],actions[j],:] += phi[next_states[j],:] - phi[states[j],:]
                counter[states[j],actions[j]] += 1
        for s in range(env.n_states):
            for a in range(env.n_actions):
                if counter[s,a]!=0:
                    A[s,a] /= counter[s,a]
        for s in range(env.n_states):
            for a in range(env.n_actions):
                if counter[s,a]!=0:
                    D[s,a,:] /= counter[s,a]
        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            for j in range(len(states)):
                numerator += np.exp(eta*A[states[j],actions[j]]) * D[states[j],actions[j]]
                denominator += np.exp(eta*A[states[j],actions[j]])
        return ((1/eta) * numerator / denominator)

    def compute_phi(self,p):
        phi = np.zeros((self.env.n_states,p))
        for k in range(self.env.n_states):
            phi[k,:] = [k,k**2,np.log(k+1)]
        return phi

    def policy_update(self):
        """Relative Entropy Policy Search using Mirror Descent"""
        p = 3    
        # initialization of the distribution
        pi = self.initialize_pi()
        policy = Policy(pi)
        #Tmax =  -100*np.log(10e-6)/(1-env.gamma)
        T = 100
        theta = [0 for i in range(p)]
        phi = self.compute_phi(p)
        
        for k in tqdm(range(self.K), desc="Iterating REPS algorithm..."):
            ##### SAMPLING
            samples = self.collect_episodes(policy=policy,horizon=T,n_episodes=self.N)

            #### OPTIMIZE
            theta = opt.fmin_bfgs(self.g,x0=theta,fprime=self.Dg,args=(self.eta,phi,samples), disp=0)

            #### COMPUTE THE NEW POLICY
            policy = self.compute_new_policy(self.eta,policy,phi,theta,samples) 
            
        self.policy = policy
        self.theta = theta
        self.phi = phi
        
    

In [22]:
REPS_model = REPS(env)
REPS_model.policy_update()

Iterating REPS algorithm...: 100%|██████████| 50/50 [06:10<00:00, 10.55s/it]


In [23]:
REPS_model.render_policy()

## DPP-RL Algorithm 
### based on the paper "Dynamic Policy Programming" in docs file

In [8]:
class DPP(RLModel):
    
    def __init__(self, env, K=50, eta=0.1):
        RLModel.__init__(self, env)
        self.K = K
        self.eta = eta

    def update_pi(self, pi, phi):
        n_states,n_actions = np.shape(pi)
        pi = np.zeros((n_states,n_actions))
        for s in range(n_states):
            a = np.argmax(phi[s,:])
            pi[s,a] = 1
        return(pi)

    def compute_Mphi(self, phi):
        n_states,n_actions = np.shape(phi)
        Mphi =  np.zeros(n_states)
        for s in range(n_states):
            numerator = 0
            denominator = 0
            for a in range(n_actions):
                numerator += np.exp(self.eta*phi[s,a]) * phi[s,a]
                denominator += np.exp(self.eta*phi[s,a])
            Mphi[s] = numerator / denominator
        return(Mphi)

    def update_phi(self, pi, phi):
        n_states,n_actions = np.shape(pi)
        Mphi = self.compute_Mphi(phi)
        for s in range(n_states):
            for a in range(n_actions):
                next_state, reward, _ = self.env.step(action=a,state=s)
                bellman_op = reward + self.env.gamma * Mphi[next_state]
                phi[s,a] += bellman_op - Mphi[s]
        return phi

    def policy_update(self):
        pi = self.initialize_pi()
        phi = np.ones((self.env.n_states, self.env.n_actions))
        
        for k in tqdm(range(self.K), desc="Iterating DPP algorithm..."):
            phi = self.update_phi(pi, phi)
            pi = self.update_pi(pi, phi)
            
        self.policy = Policy(pi)
        self.phi = phi
        

In [9]:
DPP_model = DPP(env)
DPP_model.policy_update()

Iterating DPP algorithm...: 100%|██████████| 50/50 [00:00<00:00, 1439.00it/s]


In [10]:
DPP_model.render_policy()

# TRPO

In [11]:
class TRPO(RLModel):
    def __init__(self, env, K=200, N = 100, eta=0.1):
        RLModel.__init__(self, env)
        self.K = K
        self.N = N
        self.eta = eta

    def compute_Q(self, samples, Q):
        n_states,n_actions = np.shape(Q)
        counter = np.zeros((n_states,n_actions))
        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            rewards = samples[i]['rewards']
            next_states = samples[i]['next_states']

            cumulated_rwd = 0
            for j in range(len(states)):
                t = len(states) -1 - j
                cumulated_rwd = self.env.gamma * cumulated_rwd + rewards[t]
                Q[states[t],actions[t]] += cumulated_rwd
                counter[states[t],actions[t]] += 1

        for s in range(n_states):
            for a in range(n_actions):
                if counter[s,a]!=0:
                    Q[s,a] /= counter[s,a]
        return Q


    def compute_value_function(self, Q, pi):
        n_states,n_actions = np.shape(Q)
        V = np.zeros(n_states)
        for s in range(n_states):
            for a in range(n_actions):
                V[s] += pi[s,a] * Q[s,a]
        return V

    def compute_advantage_function(self, pi, V, samples):
        n_states, n_actions = np.shape(pi)
        A = np.zeros((n_states,n_actions))
        counter = np.zeros((n_states,n_actions))

        for i in range(len(samples)):
            states = samples[i]['states']
            actions = samples[i]['actions']
            rewards = samples[i]['rewards']
            next_states = samples[i]['next_states']

            for j in range(len(states)):
                A[states[j],actions[j]] += rewards[j] + V[next_states[j]] - V[states[j]]
                counter[states[j],actions[j]] += 1

        for s in range(n_states):
            for a in range(n_actions):
                if counter[s,a]!=0:
                    A[s,a] /= counter[s,a]
        return A

    def update_pi(self, pi, A):
        n_states,n_actions = np.shape(pi)
        for s in range(n_states):
            for a in range(n_actions):
                pi[s,a] *= np.exp(self.eta * A[s,a]) 
            pi[s,:] /= np.sum(pi[s,:])
        return(pi)


    def policy_update(self):
        pi = self.initialize_pi()
        A = np.ones((env.n_states,env.n_actions))
        Q = np.ones((env.n_states,env.n_actions))
        V = np.ones(env.n_states)
        T = 100
        for k in tqdm(range(self.K), desc="Iterating TRPO algorithm..."):
            policy = Policy(pi)
            ##### SAMPLING
            samples = self.collect_episodes(policy=policy,horizon=T,n_episodes=self.N)  

            #### COMPUTE Q FUNCTION
            Q = self.compute_Q(samples,Q)

            #### COMPUTE VALUE FUNCTION
            V = self.compute_value_function(Q,pi)

            #### COMPUTE ADVANTAGE FUNCTION
            A = self.compute_advantage_function(pi,V,samples)

            #### COMPUTE THE NEW POLICY
            pi = self.update_pi(pi, A)  
            
        self.policy = Policy(pi)
        self.A = A
        self.V = V
        self.Q = Q
        

In [12]:
TRPO_model = TRPO(env)
TRPO_model.policy_update()

Iterating TRPO algorithm...: 100%|██████████| 200/200 [00:04<00:00, 55.92it/s]


In [13]:
TRPO_model.render_policy()

# A3C

In [16]:
class A3C(TRPO):
    def __init__(self, env, K=300, N = 100, eta=0.1):
        TRPO.__init__(self, env, K, N, eta)
        self.K = K
        self.N = N
        self.eta_init = eta

    def update_pi(self, pi, A):
        n_states,n_actions = np.shape(pi)
        for s in range(n_states):
            for a in range(n_actions):
                pi[s,a] = np.exp(self.eta * A[s,a]) 
            pi[s,:] /= np.sum(pi[s,:])
        return(pi)
    
    def update_eta(self):
        self.eta += self.eta_init

    def policy_update(self):
        pi = self.initialize_pi()
        A = np.ones((env.n_states,env.n_actions))
        Q = np.ones((env.n_states,env.n_actions))
        V = np.ones(env.n_states)
        T = 100
        for k in tqdm(range(self.K), desc="Iterating A3C algorithm..."):
            ##### UPDATING ETA
            self.update_eta()
            
            ##### CONSTRUCTING POLICY
            policy = Policy(pi)
            
            ##### SAMPLING
            samples = self.collect_episodes(policy=policy,horizon=T,n_episodes=self.N)  

            #### COMPUTE Q FUNCTION
            Q = self.compute_Q(samples,Q)

            #### COMPUTE VALUE FUNCTION
            V = self.compute_value_function(Q,pi)

            #### COMPUTE ADVANTAGE FUNCTION
            A = self.compute_advantage_function(pi,V,samples)

            #### COMPUTE THE NEW POLICY
            pi = self.update_pi(pi, A)  
            
        self.policy = Policy(pi)
        self.A = A
        self.V = V
        self.Q = Q
        

In [17]:
A3C_model = A3C(env)
A3C_model.policy_update()

Iterating A3C algorithm...: 100%|██████████| 300/300 [00:13<00:00, 22.03it/s]


In [18]:
A3C_model.render_policy()

In [ ]:
# TODO : Plot functions for reward and sensitivity to eta